In [1]:
from tqdm import tqdm

import numpy as np
import numpy.random as random

import math

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torch.nn.utils.rnn import pad_sequence

import io

In [2]:
def load_embeddings(fname, get_embeddings=True, get_w2i=False, get_i2w=False, skip_first_line=True):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    
    if skip_first_line:
        fin.readline()
    
    num_embeddings = 0
    
    word2idx = {}
    idx2word = {}

    embeddings = []

    for line in fin:
        line = line.rstrip().split(' ')
        
        if get_w2i:
            word2idx[line[0]] = num_embeddings
        if get_i2w:
            idx2word[num_embeddings] = line[0]
        if get_embeddings:       
            embeddings.append([float(num) for num in line[1:]])
        
        num_embeddings += 1
        
        
    return torch.FloatTensor(embeddings), word2idx, idx2word

In [3]:
word2idx = load_embeddings('../embeddings/wiki-news-300d-1M.vec', get_embeddings=False, get_w2i=True)[1]

In [4]:
class MultiplicativeAttention(nn.Module):
      
    def __init__(self, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        
        attn = torch.matmul(q , k.transpose(-2, -1) / math.sqrt(q.size(-1)))
        
        if mask is not None:
            attn = attn.masked_fill(mask.unsqueeze(1) == 1, -1e9)
        
        attn = self.dropout(F.softmax(attn, dim=-1))        
        res = torch.matmul(attn, v)

        return res, attn

class AdditiveSelfAttention(nn.Module):
    
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        
        self.w = nn.Linear(d_model, d_model)
        self.q = torch.nn.Parameter(torch.FloatTensor(d_model).uniform_(-0.1, 0.1))
    
    def forward(self, x, mask=None):
        attn = torch.tanh(self.dropout(self.w(x)))        
        attn = torch.matmul(attn, self.q)
        
        if mask is not None:
            attn = attn.masked_fill(mask == 1, -1e9)
        
        attn = self.dropout(F.softmax(attn, dim=-1))

        
        res = torch.einsum('ijk, ij->ik', x, attn)
        return res, attn

    
class MultiHeadAttention(nn.Module):
    
    def __init__(self, d_model, num_heads, d_qk, d_v, track_agreement=False, dropout=0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_qk = d_qk
        self.d_v = d_v
        
        self.num_heads = num_heads
        
        self.dropout = nn.Dropout(dropout)
        
        self.w_q = nn.Linear(d_model, num_heads * d_qk, bias=False)
        self.w_k = nn.Linear(d_model, num_heads * d_qk, bias=False)
        self.w_v = nn.Linear(d_model, num_heads * d_v, bias=False)
        
        self.w_fc = nn.Linear(num_heads * d_v, d_model, bias=False)
        
        self.attention = MultiplicativeAttention(dropout=dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
    
        self.track_agreement = track_agreement
        self.v_agreement = 0

    def forward(self, q, k, v, mask=None):     
        batch_size = q.shape[0]
        seq_size = q.shape[1]
        
        q_proj = self.w_q(q).view(q.shape[0], q.shape[1], self.num_heads, self.d_qk)
        k_proj = self.w_k(k).view(k.shape[0], k.shape[1], self.num_heads, self.d_qk)
        v_proj = self.w_v(v).view(v.shape[0], v.shape[1], self.num_heads, self.d_v) 

        if self.track_agreement:
            self.v_agreement += torch.einsum('bshd, bsnd->', F.normalize(v_proj, dim=3), F.normalize(v_proj, dim=3)) / self.num_heads**2

        if mask is None:
            q, attn = self.attention(q_proj.transpose(1, 2), k_proj.transpose(1, 2), v_proj.transpose(1, 2))
        else:
            q, attn = self.attention(q_proj.transpose(1, 2), k_proj.transpose(1, 2), v_proj.transpose(1, 2), mask.unsqueeze(1))
        
        q = q.transpose(1, 2).contiguous()
        q = q.view(batch_size, seq_size, -1)

        q = self.dropout(self.w_fc(q))

        q = self.layer_norm(q)
        
        return q, attn

    def clear_agreement(self):
        self.v_agreement = 0

class NonlinearFF(nn.Module):
    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid)
        self.w_2 = nn.Linear(d_hid, d_in)
        self.layer_norm = nn.LayerNorm(d_in, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.w_2(F.relu(self.w_1(x)))
        x = self.dropout(x)

        x = self.layer_norm(x)

        return x
    
class TitleEmbedding(nn.Module):
    def __init__(self, num_embeddings, d_model, num_heads, d_qk, d_v, d_hid=None, embeddings=None, track_agreement=False, padding_idx=0, dropout=0.1):
        super().__init__()

        if embeddings is not None:
            self.embeddings = nn.Embedding.from_pretrained(embeddings, freeze=False, sparse=True, padding_idx=padding_idx)
        else:
            self.embeddings = nn.Embedding(num_embeddings, d_model, sparse=True, padding_idx=0)
            
        self.mh_attn = MultiHeadAttention(d_model, num_heads, d_qk, d_v, track_agreement=track_agreement, dropout=dropout)
        self.nff = NonlinearFF(d_model, d_hid if d_hid is not None else d_model * 4, dropout=dropout)
        self.add_attn = AdditiveSelfAttention(d_model, dropout=dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        
        self.padding_idx = padding_idx
        
    def forward(self, title):    
        mask = (title == self.padding_idx).byte()
        
        q = k = v = self.embeddings(title)
        title, attn = self.mh_attn(q, k ,v, mask=mask)
        
        title = self.nff(title)
        title, add_attn = self.add_attn(title, mask=mask)
        
        title = self.layer_norm(title)
        
        return title
    
    def load_embeddings(embeddings):
        self.embeddings = nn.Embedding.from_pretrained(embeddings, freeze=False, sparse=True)
    
    


In [5]:
class NegativeSamplingLoss(nn.Module):
    def __init__(self, sample_pool, embedding, num_samples, device='cpu'):
        super(NegativeSamplingLoss, self).__init__()
        
        self.sample_pool = sample_pool
        self.num_samples = num_samples
        
        self.embedding = embedding
        
        self.device = device
        
    def forward(self, x, y):
        batch_size = x.shape[0]
        
        sample_indices = torch.FloatTensor(batch_size * self.num_samples).uniform_(0, len(self.sample_pool) - 1).long()        
        sample_indices, tmp = torch.broadcast_tensors(sample_indices.unsqueeze(1), 
                                                      torch.arange(batch_size * self.num_samples * self.sample_pool.shape[1])
                                                      .view(batch_size * self.num_samples, self.sample_pool.shape[1]))
        
        n = torch.gather(self.sample_pool, 0, sample_indices).to(self.device)
        
        x = self.embedding(x)
        y = self.embedding(y)
        n = self.embedding(n).neg()
        
        target_loss = (torch.einsum('ij,ij->i', x, y) / x.shape[-1]).sigmoid().mean() 
        noise_loss = (torch.einsum('ij, ikj->ik', x, n.view(batch_size, self.num_samples, n.shape[-1])) / (x.shape[-1])).sigmoid().log().sum(1).mean()
        
        return -(target_loss + noise_loss)        

In [6]:
dataset = torch.load('../data/dataset.pt')

In [8]:
device = torch.device('cuda:0')

title_embedding = torch.load('../models/model_reg_init.pt')
nss = NegativeSamplingLoss(dataset.tensors[1], title_embedding, 5, device=device).to(device)

In [ ]:
#torch.save(title_embedding, '../models/model_reg_init.pt')

In [ ]:
#title_embedding.to(torch.device('cpu'))
#nss.to(torch.device('cpu'))

In [11]:
class MultipleOptimizer:
    def __init__(self, *op):
        self.optimizers = op

    def zero_grad(self):
        for op in self.optimizers:
            op.zero_grad()

    def step(self):
        for op in self.optimizers:
            op.step()

sparse_params = []
dense_params = []

for name, param in title_embedding.named_parameters():
    if name == 'embeddings.weight':
        sparse_params.append(param)
    else:
        dense_params.append(param)
        
opt_dense = torch.optim.Adam(dense_params, lr=1e-4, weight_decay=0.0001)
opt_sparse = torch.optim.SparseAdam(sparse_params, lr=1e-4)

optimizer = MultipleOptimizer(opt_sparse, opt_dense)

train_loader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=64)

In [12]:
losses = []

In [13]:
num_epochs = 5
batch_size = 64
reg_coeff = 1.

for epoch in range(num_epochs):
        epoch_loss = 0        
        
        for idx, batch in enumerate(train_loader):
            x, y = batch[0].to(device), batch[1].to(device)
            
            optimizer.zero_grad()
            loss = nss(x, y)
            
            loss += reg_coeff * title_embedding.mh_attn.v_agreement
            title_embedding.mh_attn.clear_agreement()
            
            if loss.isnan() or loss.isinf():
                continue
            
            loss.backward()
            optimizer.step()
        
            epoch_loss += loss.item()
            
            if (idx + 1) % batch_size == 0:
                print('loss at batch {}: {}'.format((idx+1), loss.item()))
                losses.append(loss.item())
                
            
            if (idx + 1) % 16384 == 0:
                torch.save(title_embedding, '../models/model{}_{}.pt'.format(epoch, (int)(idx+1) / 16384))
            
        print('loss at end of epoch {}: {}'.format(epoch, epoch_loss/len(train_loader)))
        
        with open('../models/model{}.pt'.format(epoch+1), 'wb') as of:
            torch.save(title_embedding, of)

loss at batch 64: 66.75123596191406
loss at batch 128: 46.99741744995117
loss at batch 192: 33.35817337036133
loss at batch 256: 26.671594619750977
loss at batch 320: 21.124557495117188
loss at batch 384: 17.891103744506836
loss at batch 448: 14.759486198425293
loss at batch 512: 13.05744457244873
loss at batch 576: 11.62479019165039
loss at batch 640: 10.634286880493164
loss at batch 704: 9.653124809265137
loss at batch 768: 8.771020889282227
loss at batch 832: 8.396596908569336
loss at batch 896: 7.720708847045898
loss at batch 960: 7.804649353027344
loss at batch 1024: 7.037797927856445
loss at batch 1088: 6.838610649108887
loss at batch 1152: 6.1069536209106445
loss at batch 1216: 6.123982906341553
loss at batch 1280: 5.810311794281006
loss at batch 1344: 5.484839916229248
loss at batch 1408: 5.290788173675537
loss at batch 1472: 5.220921993255615
loss at batch 1536: 5.000507354736328
loss at batch 1600: 5.039775371551514
loss at batch 1664: 4.806284427642822
loss at batch 1728: 4.

loss at batch 13696: 2.816551923751831
loss at batch 13760: 2.8801352977752686
loss at batch 13824: 2.9046051502227783
loss at batch 13888: 2.9112181663513184
loss at batch 13952: 2.855391502380371
loss at batch 14016: 2.899574041366577
loss at batch 14080: 2.8524420261383057
loss at batch 14144: 2.88330340385437
loss at batch 14208: 2.92085337638855
loss at batch 14272: 2.88553524017334
loss at batch 14336: 2.8869450092315674
loss at batch 14400: 2.81593656539917
loss at batch 14464: 2.8988704681396484
loss at batch 14528: 2.7620739936828613
loss at batch 14592: 2.8919174671173096
loss at batch 14656: 3.039677619934082
loss at batch 14720: 2.925189733505249
loss at batch 14784: 2.9111697673797607
loss at batch 14848: 2.8044509887695312
loss at batch 14912: 2.9497694969177246
loss at batch 14976: 2.844222068786621
loss at batch 15040: 2.9476261138916016
loss at batch 15104: 2.8522894382476807
loss at batch 15168: 2.8199918270111084
loss at batch 15232: 2.910844087600708
loss at batch 1

loss at batch 27008: 2.7816121578216553
loss at batch 27072: 2.727949857711792
loss at batch 27136: 2.8570003509521484
loss at batch 27200: 2.8516318798065186
loss at batch 27264: 2.810133218765259
loss at batch 27328: 2.765998125076294
loss at batch 27392: 2.8323867321014404
loss at batch 27456: 2.8149282932281494
loss at batch 27520: 2.7756378650665283
loss at batch 27584: 2.8505735397338867
loss at batch 27648: 2.8033416271209717
loss at batch 27712: 2.823866128921509
loss at batch 27776: 2.848714590072632
loss at batch 27840: 2.7931625843048096
loss at batch 27904: 2.9010963439941406
loss at batch 27968: 2.7272305488586426
loss at batch 28032: 2.766465187072754
loss at batch 28096: 2.894645929336548
loss at batch 28160: 2.7393741607666016
loss at batch 28224: 2.7678189277648926
loss at batch 28288: 2.773531198501587
loss at batch 28352: 2.8636231422424316
loss at batch 28416: 2.844917058944702
loss at batch 28480: 2.769022226333618
loss at batch 28544: 2.847860336303711
loss at bat

loss at batch 40320: 2.8124825954437256
loss at batch 40384: 2.8456873893737793
loss at batch 40448: 2.7978808879852295
loss at batch 40512: 2.763720750808716
loss at batch 40576: 2.817284345626831
loss at batch 40640: 2.7488749027252197
loss at batch 40704: 2.700432062149048
loss at batch 40768: 2.791775941848755
loss at batch 40832: 2.9596965312957764
loss at batch 40896: 2.7570412158966064
loss at batch 40960: 2.791029214859009
loss at batch 41024: 2.91772198677063
loss at batch 41088: 2.717960834503174
loss at batch 41152: 2.8026070594787598
loss at batch 41216: 2.7738583087921143
loss at batch 41280: 2.881950855255127
loss at batch 41344: 2.7257463932037354
loss at batch 41408: 2.7600693702697754
loss at batch 41472: 2.782583236694336
loss at batch 41536: 2.771139144897461
loss at batch 41600: 2.9173591136932373
loss at batch 41664: 2.7050440311431885
loss at batch 41728: 2.819368362426758
loss at batch 41792: 2.669616222381592
loss at batch 41856: 2.784446954727173
loss at batch 

loss at batch 53632: 2.915766477584839
loss at batch 53696: 2.766934633255005
loss at batch 53760: 2.825538396835327
loss at batch 53824: 2.7493653297424316
loss at batch 53888: 2.7898037433624268
loss at batch 53952: 2.7268104553222656
loss at batch 54016: 2.7507729530334473
loss at batch 54080: 2.773775100708008
loss at batch 54144: 2.6594395637512207
loss at batch 54208: 2.846226692199707
loss at batch 54272: 2.761691093444824
loss at batch 54336: 2.692112445831299
loss at batch 54400: 2.6366827487945557
loss at batch 54464: 2.8336589336395264
loss at batch 54528: 2.802342414855957
loss at batch 54592: 2.630979537963867
loss at batch 54656: 2.6844379901885986
loss at batch 54720: 2.6554057598114014
loss at batch 54784: 2.784353256225586
loss at batch 54848: 2.659458637237549
loss at batch 54912: 2.7063894271850586
loss at batch 54976: 2.699251651763916
loss at batch 55040: 2.688809394836426
loss at batch 55104: 2.715444564819336
loss at batch 55168: 2.6963589191436768
loss at batch 

...

loss at batch 24704: 1.3474465608596802
loss at batch 24768: 1.7657803297042847
loss at batch 24832: 1.4091780185699463
loss at batch 24896: 1.3706903457641602
loss at batch 24960: 1.499097466468811
loss at batch 25024: 1.295345664024353
loss at batch 25088: 1.6962002515792847
loss at batch 25152: 1.3895957469940186
loss at batch 25216: 1.2811427116394043
loss at batch 25280: 1.246339201927185
loss at batch 25344: 1.293062448501587
loss at batch 25408: 1.5050016641616821
loss at batch 25472: 1.4188240766525269
loss at batch 25536: 1.4606834650039673
loss at batch 25600: 1.4388186931610107
loss at batch 25664: 1.3177781105041504
loss at batch 25728: 1.785177230834961
loss at batch 25792: 1.5129873752593994
loss at batch 25856: 1.3706518411636353
loss at batch 25920: 2.4910531044006348
loss at batch 25984: 1.231197476387024
loss at batch 26048: 1.6261509656906128
loss at batch 26112: 1.51939058303833
loss at batch 26176: 1.4553358554840088
loss at batch 26240: 1.5003184080123901
loss at 

loss at batch 37952: 1.1831907033920288
loss at batch 38016: 1.4776099920272827
loss at batch 38080: 1.510644555091858
loss at batch 38144: 2.16936993598938
loss at batch 38208: 1.5486377477645874
loss at batch 38272: 1.2005616426467896
loss at batch 38336: 1.4735326766967773
loss at batch 38400: 1.6698955297470093
loss at batch 38464: 1.4448119401931763
loss at batch 38528: 1.4077224731445312
loss at batch 38592: 1.3983525037765503
loss at batch 38656: 1.2728406190872192
loss at batch 38720: 1.3324402570724487
loss at batch 38784: 1.3455169200897217
loss at batch 38848: 1.5780043601989746
loss at batch 38912: 1.3684868812561035
loss at batch 38976: 1.534103274345398
loss at batch 39040: 1.6419296264648438
loss at batch 39104: 1.5289592742919922
loss at batch 39168: 1.364574909210205
loss at batch 39232: 1.5066109895706177
loss at batch 39296: 1.451624870300293
loss at batch 39360: 1.3010140657424927
loss at batch 39424: 1.3691118955612183
loss at batch 39488: 1.9857182502746582
loss a

loss at batch 51200: 2.1877524852752686
loss at batch 51264: 1.3580201864242554
loss at batch 51328: 1.2527623176574707
loss at batch 51392: 1.2133424282073975
loss at batch 51456: 1.1486387252807617
loss at batch 51520: 1.2204854488372803
loss at batch 51584: 2.4216394424438477
loss at batch 51648: 1.610790491104126
loss at batch 51712: 1.5114918947219849
loss at batch 51776: 0.9596057534217834
loss at batch 51840: 1.315780758857727
loss at batch 51904: 1.2247287034988403
loss at batch 51968: 1.555325984954834
loss at batch 52032: 1.3328242301940918
loss at batch 52096: 1.6013100147247314
loss at batch 52160: 1.442153811454773
loss at batch 52224: 1.6122807264328003
loss at batch 52288: 1.3478187322616577
loss at batch 52352: 1.7553346157073975
loss at batch 52416: 1.3088953495025635
loss at batch 52480: 2.396237850189209
loss at batch 52544: 1.535386562347412
loss at batch 52608: 1.3732677698135376
loss at batch 52672: 1.096889853477478
loss at batch 52736: 1.2309954166412354
loss at

loss at batch 64448: 1.0541563034057617
loss at batch 64512: 1.0542476177215576
loss at batch 64576: 1.4600673913955688
loss at batch 64640: 1.3554391860961914
loss at batch 64704: 1.3420755863189697
loss at batch 64768: 1.3126546144485474
loss at batch 64832: 1.5440335273742676
loss at batch 64896: 1.6679800748825073
loss at batch 64960: 1.2868483066558838
loss at batch 65024: 1.3626399040222168
loss at batch 65088: 1.1217142343521118
loss at batch 65152: 1.021314024925232
loss at batch 65216: 1.5684555768966675
loss at batch 65280: 1.5149649381637573
loss at batch 65344: 1.1531236171722412
loss at batch 65408: 1.2148257493972778
loss at batch 65472: 1.2140721082687378
loss at batch 65536: 1.2466192245483398
loss at batch 65600: 1.4432514905929565
loss at batch 65664: 1.4813655614852905
loss at batch 65728: 1.1811723709106445
loss at batch 65792: 1.2218519449234009
loss at batch 65856: 1.2808061838150024
loss at batch 65920: 1.310281753540039
loss at batch 65984: 1.2263480424880981
lo

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.xscale('log')
plt.plot(range(len(losses)), losses,'bx')

In [ ]:
title_embedding.mh_attn

In [ ]:
dummy_title = torch.LongTensor([1, 9, 17])
padded_title = torch.LongTensor([1, 9, 17, 0])

w_embeddings = nn.Embedding.from_pretrained(embeddings, freeze=False, sparse=True, padding_idx=0)
w_embeddings.weight.data[0] = torch.zeros(300)
mult_attn = MultiplicativeAttention()
add_attn = AdditiveSelfAttention(300)
mh_attn = MultiHeadAttention(300, 12, 25, 25)
nff = NonlinearFF(300, 25)

torch.set_printoptions(linewidth=120,threshold=100)
mult_attn.eval()
mh_attn.eval()
nff.eval()
add_attn.eval()
mult_attn.eval()
title_embedding.eval()

def transform(title, mask=None): 
    z = title
    s = mask
    print('title: {}'.format(title))
    
    q = k = v = w_embeddings(title)
    
    print('embedded title: \n{}'.format(q))
    print(q.shape)
    
    title, attn = mh_attn(q, k, v, mask)
    
    print('transformed title: \n{}'.format(title))
    print(title.shape)
    
    title = nff(title)
    
    print('title after nonlinear ff: \n {}'.format(title))
    print(title.shape)
    
    title, importance = add_attn(title, mask)
    
    print('importance of each word: \n {}'.format(importance))
    print(importance.shape)
    
    print('title after everything: \n {}'.format(title))
    print(title.shape)
    
    
    y = title_embedding(z, mask=s)
    print(y)
    print(y.shape)

transform(torch.LongTensor([[1, 5, 7, 2, 9, 8, 0, 0], [2, 5, 5, 0, 0, 0, 0, 0]]),        
                    mask= torch.ByteTensor([[0,0,0,0,0,0,1,1],[0,0,0,1,1,1,1,1]]))